# 情感词典

In [1]:

import jieba
import numpy as np

#打开词典文件，返回列表
def open_dict(Dict = 'hahah', path=r'/'):
    path = path + '%s.txt' % Dict
    dictionary = open(path, 'r', encoding='utf-8') #py3
    #dictionary = open(path, 'r')
    dict = []
    for word in dictionary:
        word = word.strip('\n')
        dict.append(word)
    return dict



def judgeodd(num):
    if (num % 2) == 0:
        return 'even'
    else:
        return 'odd'


deny_word = open_dict(Dict = 'notdoc', path= r'dict/')
posdict = open_dict(Dict = 'positive', path= r'dict/')
negdict = open_dict(Dict = 'negative', path= r'dict/')

degree_word = open_dict(Dict = 'degreedoc', path= r'dict/')
mostdict = degree_word[degree_word.index('extreme')+1 : degree_word.index('very')]#权重4，即在情感词前乘以4
verydict = degree_word[degree_word.index('very')+1 : degree_word.index('more')]#权重3
moredict = degree_word[degree_word.index('more')+1 : degree_word.index('ish')]#权重2
ishdict = degree_word[degree_word.index('ish')+1 : degree_word.index('last')]#权重0.5

In [2]:
# def print_basics(dictl):
#     print(type(dictl))
#     print(len(dictl))
#     print(dictl[0])
#     print(dictl[1])
# print_basics(degree_word)
# print_basics(mostdict)
# print_basics(verydict)
# print_basics(moredict)
# print_basics(ishdict)

In [3]:
def sentiment_score_list(dataset):
    seg_sentence = dataset.split('。')

    count1 = []
    count2 = []
    for sen in seg_sentence: #循环遍历每一个评论
        segtmp = jieba.lcut(sen, cut_all=False)  #把句子进行分词，以列表的形式返回
        i = 0 #记录扫描到的词的位置
        a = 0 #记录情感词的位置
        poscount = 0 #积极词的第一次分值
        poscount2 = 0 #积极词反转后的分值
        poscount3 = 0 #积极词的最后分值（包括叹号的分值）
        negcount = 0
        negcount2 = 0
        negcount3 = 0
        for word in segtmp:
            if word in posdict:  # 判断词语是否是情感词
                poscount += 1
                c = 0
                for w in segtmp[a:i]:  # 扫描情感词前的程度词
                    if w in mostdict:
                        poscount *= 4.0
                    elif w in verydict:
                        poscount *= 3.0
                    elif w in moredict:
                        poscount *= 2.0
                    elif w in ishdict:
                        poscount *= 0.5
                    elif w in deny_word:
                        c += 1
                if judgeodd(c) == 'odd':  # 扫描情感词前的否定词数
                    poscount *= -1.0
                    poscount2 += poscount
                    poscount = 0
                    poscount3 = poscount + poscount2 + poscount3
                    poscount2 = 0
                else:
                    poscount3 = poscount + poscount2 + poscount3
                    poscount = 0
                a = i + 1  # 情感词的位置变化

            elif word in negdict:  # 消极情感的分析，与上面一致
                negcount += 1
                d = 0
                for w in segtmp[a:i]:
                    if w in mostdict:
                        negcount *= 4.0
                    elif w in verydict:
                        negcount *= 3.0
                    elif w in moredict:
                        negcount *= 2.0
                    elif w in ishdict:
                        negcount *= 0.5
                    elif w in degree_word:
                        d += 1
                if judgeodd(d) == 'odd':
                    negcount *= -1.0
                    negcount2 += negcount
                    negcount = 0
                    negcount3 = negcount + negcount2 + negcount3
                    negcount2 = 0
                else:
                    negcount3 = negcount + negcount2 + negcount3
                    negcount = 0
                a = i + 1
            elif word == '！' or word == '!':  ##判断句子是否有感叹号
                for w2 in segtmp[::-1]:  # 扫描感叹号前的情感词，发现后权值+2，然后退出循环
                    if w2 in posdict or negdict:
                        poscount3 += 2
                        negcount3 += 2
                        break
            i += 1 # 扫描词位置前移


            # 以下是防止出现负数的情况
            pos_count = 0
            neg_count = 0
            if poscount3 < 0 and negcount3 > 0:
                neg_count += negcount3 - poscount3
                pos_count = 0
            elif negcount3 < 0 and poscount3 > 0:
                pos_count = poscount3 - negcount3
                neg_count = 0
            elif poscount3 < 0 and negcount3 < 0:
                neg_count = -poscount3
                pos_count = -negcount3
            else:
                pos_count = poscount3
                neg_count = negcount3

            count1.append([pos_count, neg_count])
        count2.append(count1)
        count1 = []

    return count2

# def sentiment_score(senti_score_list):
#     score = []
#     for review in senti_score_list:
#         score_array = np.array(review)
#         Pos = np.sum(score_array[:, 0])
#         Neg = np.sum(score_array[:, 1])
#         AvgPos = np.mean(score_array[:, 0])
#         AvgPos = float('%.1f'%AvgPos)
#         AvgNeg = np.mean(score_array[:, 1])
#         AvgNeg = float('%.1f'%AvgNeg)
#         StdPos = np.std(score_array[:, 0])
#         StdPos = float('%.1f'%StdPos)
#         StdNeg = np.std(score_array[:, 1])
#         StdNeg = float('%.1f'%StdNeg)
#         score.append([Pos, Neg, AvgPos, AvgNeg, StdPos, StdNeg])
#         score.append
#     return score

def sentiment_score(senti_score_list):
    score = []
    for review in senti_score_list:
        score_array = np.array(review)
        Pos = np.sum(score_array[:, 0])
        Neg = np.sum(score_array[:, 1])
        score.append([Pos, Neg])
        score.append
    return score

def posneg_score(senti_score_list):
    score = []
    for review in senti_score_list:
        score_array = np.array(review)
        Pos = np.sum(score_array[:, 0])
        Neg = np.sum(score_array[:, 1])
        score.append([Pos, Neg])
        score.append
    if score[0][0] > score[0][1]:
        return 1
    else:
        return 0

In [4]:
# data = '你就是个王八蛋，混账玩意!你们的手机真不好用！非常生气，我非常郁闷！！！！'
# data2= '我好开心啊，非常非常非常高兴！今天我得了一百分，我很兴奋开心，愉快，开心'
# print(sentiment_score(sentiment_score_list(data)))
# print(sentiment_score(sentiment_score_list(data2)))
# print(posneg_score(sentiment_score_list(data2)))
# print(sentiment_score_list(data2))

# 情感评分列表

In [30]:
#生成一个对英美一条评论和弹幕的情感得分表（0，1）
######read danmu
with open("allacfundanmu_1.txt", encoding = "utf-8") as f:
    data = f.read().splitlines()
danmulist = []
danmucode = []
for eachline in data:
    oneobs = eachline.split('\t')
    if len(oneobs) == 11:
        danmucode.append(oneobs[1])
        danmulist.append(oneobs[9])

In [41]:
print(len(danmucode))
print(len(danmulist))

77971
77971


In [37]:
#######read reply1:
# rurl,page,totalPage,allCount,totalCount,cid,quoteId,content,postDate,userID,
# userName,coun_t,deep,refCount,ups,downs,nameRed,avatarFrame,isDelete,isUpDelete,
# verified,crawl_time
# reply2:
# line = [rurl,int(video_code),page,totalPage,allCount,totalCount,cid,quoteId,content,postDate,userID,userName,coun_t,deep,refCount,ups,downs,nameRed,avatarFrame,isDelete,isUpDelete,verified,crawl_time]
import re
with open("allacfunreply_1.txt", encoding = "utf-8") as f:
    data = f.read().splitlines()
replylist = []
replycode = []
for eachline in data:
    oneobs = eachline.split('\t')
    if len(oneobs) == 23:
        excode = re.findall(r'contentId=(\d+)&current',oneobs[0])[0]
        replycode.append(excode)
        replylist.append(oneobs[7])

In [50]:
print(len(replycode))
print(len(replylist))
replylist[0:10]

32000
32000


['[emot=ac,01/]up主被封禁!',
 '[emot=ac,43/]远古巨坟～坟中坟～',
 'UP，你还在a站吗[emot=ac2,55/]',
 '听说这边有远古坟，慕名而来٩(*´◒`*)۶',
 '我想知道七年后有没有人来挖我',
 '一个考古员那，来到acfun那，每天都会挖到一个优秀的古坟！！！[emot=ac3,48/]',
 '终极蛇皮坟',
 '大家挖的好快啊！[emot=ac,19/]',
 '祖坟。。。。。',
 '超级古坟  留个名']

In [6]:
import numpy as np
def posneg(docu):
    try:
        point = posneg_score(sentiment_score_list(docu))
        return point
    except:
        point = 0
        return point
vc_posneg = np.vectorize(posneg)

In [20]:
# rate the danmu
import time
start = time.clock()
#long running
sentilist = vc_posneg(danmulist)

end = time.clock()
print("runtime: %f s" % (end - start))
#would take 189s to run 77971 lines of danmu (12.6Mb)

runtime: 209.667556 s


In [45]:
#videocode, danmulist, sentitlist
print(len(danmulist))
print(len(sentilist))

77971
77971


list

In [58]:
# create a danmu_sent dataframe
# lst1 = range(100)
# lst2 = range(100)
# lst3 = range(100)
# percentile_list = pd.DataFrame(
#     {'lst1Tite': lst1,
#      'lst2Tite': lst2,
#      'lst3Tite': lst3
#     })
import pandas as pd
#danmucode, danmulist, sentilist
dansendf = pd.DataFrame(
    {'code':danmucode,
     'danmu':danmulist,
     'senti':sentilist}
)
dansendf.head(7)

,code,danmu,senti
0,66,你们只是一群伴舞的,0
1,66,tdgsgl,0
2,66,睡觉,0
3,66,这坟太远古 我不敢看,1
4,66,‎2016‎年‎4‎月‎26‎日17:02:23,0
5,66,吃我一击洛阳铲,0
6,66,就只有66。。远古巨坟,0


In [51]:
posneg_score(sentiment_score_list( '一个考古员那，来到acfun那，每天都会挖到一个优秀的古坟！！！[emot=ac3,48/]'))

1

In [52]:
# rate the reply
import time
start = time.clock()
#long running
re_sentilist = []
for rep in replylist:
    re_sentilist.append(posneg(rep))

end = time.clock()
print("runtime: %f s" % (end - start))
#would take 258s to run 32000 lines of reply (12.4Mb)

runtime: 258.696734 s


In [62]:
#create a reply_senti dataframe
#replycode, replylist, re_sentilist
repsendf = pd.DataFrame(
    {'code':replycode,
     'reply':replylist,
     'senti':re_sentilist}
)
repsendf.head(10)

,code,reply,senti
0,1171,"[emot=ac,01/]up主被封禁!",0
1,1171,"[emot=ac,43/]远古巨坟～坟中坟～",0
2,114,"UP，你还在a站吗[emot=ac2,55/]",0
3,66,听说这边有远古坟，慕名而来٩(*´◒`*)۶,0
4,66,我想知道七年后有没有人来挖我,1
5,66,"一个考古员那，来到acfun那，每天都会挖到一个优秀的古坟！！！[emot=ac3,48/]",1
6,66,终极蛇皮坟,0
7,66,"大家挖的好快啊！[emot=ac,19/]",0
8,66,祖坟。。。。。,0
9,66,超级古坟 留个名,1


In [31]:
#read the video
with open("allacvideo_1.txt", encoding = "utf-8") as f:
    data = f.read().splitlines()
danmulist = []
videocode = []
for eachline in data:
    oneobs = eachline.split('\t')
    if len(oneobs) == 18:
        danmucode.append(oneobs[1])
        videocode.append(oneobs[9])

'66'

In [ ]:
#Pic.Timeline


# 主题分析 (LDA model)

In [8]:
###########弹幕
# 1. 导入
import pandas as pd  
import jieba  
import nltk  
import jieba.posseg as pseg  
from gensim import corpora, models, similarities  
#pip install nltk, jieba, gensim
# warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")

d:\programs\python\python36\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [9]:
dandf = pd.DataFrame(danmulist,columns=["danmu"])

In [10]:
dandf.head()

,danmu
0,你们只是一群伴舞的
1,tdgsgl
2,睡觉
3,这坟太远古 我不敢看
4,‎2016‎年‎4‎月‎26‎日17:02:23


In [11]:
cont = dandf['danmu']
type(cont)

pandas.core.series.Series

In [13]:
# 2. 分词+选词  
import time
start = time.clock()
#long running
nwordall = []  
for t in cont:  
    words =pseg.cut(t)  
    nword = ['']  
    for w in words:  
#         if((w.flag == 'n'or w.flag == 'v' or w.flag == 'a') and len(w.word)>1):  
        if(len(w.word)>1):  
            nword.append(w.word)  
    nwordall.append(nword) 
end = time.clock()
print("runtime: %f s" % (end - start))
#would take 77s to run 77971 lines of danmu (12.6Mb)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\l\AppData\Local\Temp\jieba.cache
Loading model cost 1.002 seconds.
Prefix dict has been built succesfully.


runtime: 82.098686 s


In [14]:
nwordall[0:10]

[['', '你们', '只是', '一群', '伴舞'],
 ['', 'tdgsgl'],
 ['', '睡觉'],
 ['', '远古', '不敢'],
 ['', '2016', '26', '17', '02', '23'],
 ['', '我一', '洛阳'],
 ['', '只有', '66', '远古'],
 ['', '在线', '对不起'],
 ['', '在线', '偶像', '大师'],
 ['', '爆音']]

In [15]:
# 3. 选择后的词生成字典  
start = time.clock()

dictionary = corpora.Dictionary(nwordall)  
# 生成语料库   
corpus = [dictionary.doc2bow(text) for text in nwordall]  
#tfidf加权  
tfidf = models.TfidfModel(corpus)  
corpus_tfidf = tfidf[corpus]  

end = time.clock()
print("runtime: %f s" % (end - start))
#would take 1s to run 77971 lines of danmu (12.6Mb)

runtime: 1.262779 s


In [16]:
# print (dictionary.token2id)
# print (tfidf) 
# for doc in corpus_tfidf:  
#      print (doc) 

In [72]:
# 4. 主题模型lda，可用于降维  
start = time.clock()

#lda计算，每块20000条记录，提取50个主题  
lda = models.ldamodel.LdaModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=50,     update_every=1, chunksize=10000, passes=1)  
for i in range(0,3):  
    print (lda.print_topics(i)[0])  
#lda全部数据建模，提取100个主题  
#lda = models.ldamodel.LdaModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=100, update_every=0, passes=20)  

end = time.clock()
print("runtime: %f s" % (end - start))
#would take 50s to run 77971 lines of danmu (12.6Mb)

(19, '0.048*"color" + 0.034*"继续" + 0.027*"不会" + 0.026*"消失" + 0.021*"nbsp" + 0.018*"13" + 0.015*"TMD" + 0.013*"感谢" + 0.012*"NTR" + 0.012*"专业"')
(44, '0.032*"下载" + 0.021*"ac" + 0.020*"作者" + 0.019*"emot" + 0.016*"07" + 0.015*"就是" + 0.015*"福利" + 0.014*"高清" + 0.013*"17" + 0.013*"养肥"')
(28, '0.082*".." + 0.044*"我们" + 0.035*"一直" + 0.030*"AC" + 0.021*"一发" + 0.018*"首页" + 0.016*"53" + 0.015*"有个" + 0.014*"而已" + 0.012*"正好"')
runtime: 22.999903 s


In [18]:
# doc_lda = lda[corpus_tfidf]  
# doc_lda

In [29]:
#5. word2vec 词向量化，可用于比较词相似度，寻找对应关系，词聚类  
start = time.clock()


#sentences = models.word2vec.LineSentence(nwordall)  
#size为词向量维度数,windows窗口范围,min_count频数小于5的词忽略,workers是线程数  
model = models.word2vec.Word2Vec(nwordall, size=100, window=5, min_count=5, workers=4)  
model.save("F:\word2vecmodels") #如果建模速度慢的话，建议保存，后续直接调用  
model = models.word2vec.Word2Vec.load("F:\word2vecmodels")  
# print (model[u'童年'])  
#向量表示  
# sim = model.most_similar(positive=[u'童年', u'传说'])  
sim = model.most_similar(positive=[u'童年'])  

#相近词  
for s in sim:  
    print ("word:%s,similar:%s " %(s[0],s[1]))  
print("#################")
sim = model.most_similar(positive=[u'传说']) 
for s in sim:  
    print ("word:%s,similar:%s " %(s[0],s[1]))  
#找到“童年”和”传说“相似度高的词  

end = time.clock()
print("runtime: %f s" % (end - start))

word:不好,similar:0.9994120597839355 
word:刚刚,similar:0.9992666840553284 
word:肯定,similar:0.9991625547409058 
word:不管,similar:0.9991371631622314 
word:衣服,similar:0.9990770220756531 
word:然后,similar:0.9990596175193787 
word:孩子,similar:0.9990497827529907 
word:看见,similar:0.9990304708480835 
word:演员,similar:0.9989286065101624 
word:里面,similar:0.9988921284675598 
#################
word:生活,similar:0.9994906187057495 
word:100,similar:0.9994415044784546 
word:小时,similar:0.9993937015533447 
word:路过,similar:0.9993805885314941 
word:PK,similar:0.9993515014648438 
word:第二,similar:0.9993427991867065 
word:系统,similar:0.9993183612823486 
word:目测,similar:0.9993164539337158 
word:ac,similar:0.9992858171463013 
word:60,similar:0.9992822408676147 
runtime: 1.489941 s


In [ ]:
# sim = model.most_similar(positive=[u'童年'])  
# for s in sim:  
#     print ("word:%s,similar:%s " %(s[0],s[1]))  
# sim = model.most_similar(positive=[u'传说'])  
# for s in sim:  
#     print ("word:%s,similar:%s " %(s[0],s[1])) 
# sim = model.most_similar(positive=[u'机智'])  
# for s in sim:  
#     print ("word:%s,similar:%s " %(s[0],s[1]))  
# sim = model.most_similar(positive=[u'QAQ'])  
# for s in sim:  
#     print ("word:%s,similar:%s " %(s[0],s[1]))  

In [63]:

###########评论
import pandas as pd  
import jieba  
import nltk  
import jieba.posseg as pseg  
from gensim import corpora, models, similarities  
repdf = pd.DataFrame(replylist,columns=["reply"])
repdf.head()

,reply
0,"[emot=ac,01/]up主被封禁!"
1,"[emot=ac,43/]远古巨坟～坟中坟～"
2,"UP，你还在a站吗[emot=ac2,55/]"
3,听说这边有远古坟，慕名而来٩(*´◒`*)۶
4,我想知道七年后有没有人来挖我


In [64]:
cont = repdf['reply']
type(cont)

pandas.core.series.Series

In [65]:
# 2. 分词+选词  
import time
start = time.clock()
#long running
nwordall = []  
for t in cont:  
    words =pseg.cut(t)  
    nword = ['']  
    for w in words:  
#         if((w.flag == 'n'or w.flag == 'v' or w.flag == 'a') and len(w.word)>1):  
        if(len(w.word)>1):  
            nword.append(w.word)  
    nwordall.append(nword) 
end = time.clock()
print("runtime: %f s" % (end - start))
#would take 107s to run 32000 lines of reply (12.4Mb)

runtime: 107.742172 s


In [66]:
# 3. 选择后的词生成字典  
start = time.clock()

dictionary = corpora.Dictionary(nwordall)  
# 生成语料库   
corpus = [dictionary.doc2bow(text) for text in nwordall]  
#tfidf加权  
tfidf = models.TfidfModel(corpus)  
corpus_tfidf = tfidf[corpus]  

end = time.clock()
print("runtime: %f s" % (end - start))
#would take 1s to run 32000 lines of reply (12.4Mb)

runtime: 0.625352 s


In [73]:
# 4. 主题模型lda，可用于降维  
start = time.clock()

#lda计算，每块10000条记录，提取50个主题  
lda = models.ldamodel.LdaModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=50,     update_every=1, chunksize=1000, passes=1)  
for i in range(0,3):  
    print (lda.print_topics(i)[0])  
#lda全部数据建模，提取100个主题  
#lda = models.ldamodel.LdaModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=100, update_every=0, passes=20)  
end = time.clock()
print("runtime: %f s" % (end - start))
#would take 23s to run 32000 lines of reply (12.4Mb)

(10, '0.055*"压力" + 0.033*"解释" + 0.021*"很大" + 0.018*"有没有" + 0.018*"一般" + 0.014*"这里" + 0.013*"片子" + 0.012*"洗脑" + 0.012*"效果" + 0.010*"战斗"')
(31, '0.115*"BGM" + 0.043*"来看" + 0.034*"碉堡" + 0.024*"百合" + 0.024*"记得" + 0.017*"2015" + 0.017*"07" + 0.016*"电视" + 0.015*"接受" + 0.013*"ED"')
(4, '0.082*"红名" + 0.057*"可以" + 0.054*"最后" + 0.033*"OP" + 0.031*"哪里" + 0.031*"15" + 0.027*"ais" + 0.026*"23" + 0.025*"18" + 0.024*"试试"')
runtime: 26.338470 s


In [77]:
#5. word2vec 词向量化，可用于比较词相似度，寻找对应关系，词聚类  

#sentences = models.word2vec.LineSentence(nwordall)  
#size为词向量维度数,windows窗口范围,min_count频数小于5的词忽略,workers是线程数  
model = models.word2vec.Word2Vec(nwordall, size=100, window=5, min_count=5, workers=4)  
model.save("F:\word2vecmodels") #如果建模速度慢的话，建议保存，后续直接调用  
model = models.word2vec.Word2Vec.load("F:\word2vecmodels")  
# print (model[u'童年'])  
#向量表示  
# sim = model.most_similar(positive=[u'童年', u'传说'])  
sim = model.most_similar(positive=[u'童年'])  

#相近词  
print("#################与童年主题相近的词汇")

for s in sim:  
    print ("word:%s,similar:%s " %(s[0],s[1]))  
sim = model.most_similar(positive=[u'传说']) 
print("#################与传说相似的词汇")

for s in sim:  
    print ("word:%s,similar:%s " %(s[0],s[1]))  
#找到“童年”和”传说“相似度高的词  


#################与童年主题相近的词汇
word:春晚,similar:0.9987530708312988 
word:CG,similar:0.9984016418457031 
word:除了,similar:0.9983508586883545 
word:国产,similar:0.998162031173706 
word:某些,similar:0.9980917572975159 
word:甚至,similar:0.9980370998382568 
word:上海,similar:0.9979149103164673 
word:看起来,similar:0.9977549314498901 
word:法师,similar:0.997751772403717 
word:半年,similar:0.997745931148529 
#################与传说相似的词汇
word:听说,similar:0.9863263964653015 
word:尼玛,similar:0.9858675003051758 
word:泥煤,similar:0.9817530512809753 
word:真的假,similar:0.981099009513855 
word:真是,similar:0.9806280136108398 
word:同一个,similar:0.9799231886863708 
word:老子,similar:0.9797074198722839 
word:难道,similar:0.9774042367935181 
word:干事,similar:0.9749318957328796 
word:红色,similar:0.9746084213256836 
